In [615]:
from splinter import Browser
from splinter.exceptions import ElementDoesNotExist
from bs4 import BeautifulSoup
import requests
import re
import time
import pandas as pd

In [616]:
executable_path = {'executable_path': 'chromedriver.exe'}
browser = Browser('chrome', **executable_path, headless=False)
time.sleep(3)

In [598]:
url = 'https://mars.nasa.gov/news/'
browser.visit(url)
time.sleep(3)

In [599]:
html = browser.html
soup = BeautifulSoup(html,'html.parser')
articles = soup.find_all('div',class_='list_text')

title_list=[]
para_list=[]

for article in articles:
    title = article.find('a').text.strip()
    title_list.append(title)
    para = article.find('div',class_='article_teaser_body').text.strip()
    para_list.append(para)
    print('______________')
    print(title)
    print(para)

print(title_list)
print(para_list)

______________
NASA Readies Perseverance Mars Rover's Earthly Twin
Did you know NASA's next Mars rover has a nearly identical sibling on Earth for testing? Even better, it's about to roll for the first time through a replica Martian landscape.
______________
NASA Engineers Checking InSight's Weather Sensors
An electronics issue is suspected to be preventing the sensors from sharing their data about Mars weather with the spacecraft.
______________
Follow NASA's Perseverance Rover in Real Time on Its Way to Mars
A crisply rendered web application can show you where the agency's Mars 2020 mission is right now as it makes its way to the Red Planet for a Feb. 18, 2021, landing.
______________
NASA Establishes Board to Initially Review Mars Sample Return Plans
The board will assist with analysis of current plans and goals for one of the most difficult missions humanity has ever undertaken.
______________
NASA's Ingenuity Mars Helicopter Recharges Its Batteries in Flight
Headed to the Red Pla

In [583]:
url = 'https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars'
browser.visit(url)
time.sleep(3)

In [584]:
browser.click_link_by_id('full_image')
time.sleep(3)

In [585]:
html = browser.html
soup = BeautifulSoup(html,'html.parser')
image_list=[]
descrip_list=[]
next = '//a[@class="fancybox-nav fancybox-next"]'

for x in range(1):

    image = soup.article.a['data-fancybox-href']
    featured_image_url = (f'https://www.jpl.nasa.gov{image}')
    image_list.append(image_path)
    descrip =soup.h1.text.strip()
    descrip_list.append(descrip)
    image_path = (f'https://www.jpl.nasa.gov{image}')
    print(featured_image_url)
    print(descrip)
    browser.find_by_xpath(next).first.click()
    time.sleep(3)

print(image_list)
print(descrip_list)

https://www.jpl.nasa.gov/spaceimages/images/mediumsize/PIA17564_ip.jpg
March of Asteroids Across Dying Star
['https://www.jpl.nasa.gov/spaceimages/images/mediumsize/PIA18846_ip.jpg']
['March of Asteroids Across Dying Star']


In [587]:
url = 'https://space-facts.com/mars/'

In [588]:
tables = pd.read_html(url)
tables

[                      0                              1
 0  Equatorial Diameter:                       6,792 km
 1       Polar Diameter:                       6,752 km
 2                 Mass:  6.39 × 10^23 kg (0.11 Earths)
 3                Moons:            2 (Phobos & Deimos)
 4       Orbit Distance:       227,943,824 km (1.38 AU)
 5         Orbit Period:           687 days (1.9 years)
 6  Surface Temperature:                   -87 to -5 °C
 7         First Record:              2nd millennium BC
 8          Recorded By:           Egyptian astronomers,
   Mars - Earth Comparison             Mars            Earth
 0               Diameter:         6,779 km        12,742 km
 1                   Mass:  6.39 × 10^23 kg  5.97 × 10^24 kg
 2                  Moons:                2                1
 3      Distance from Sun:   227,943,824 km   149,598,262 km
 4         Length of Year:   687 Earth days      365.24 days
 5            Temperature:     -87 to -5 °C      -88 to 58°C,
           

In [589]:
 type(tables) 

list

In [594]:
df = tables[0]
df.rows = ['Equatorial Diameter', 'Polar Diameter', 'Mass', 'Moons', 
              'Orbit Distance', 'Orbit Period', 'Surface Temperature', 'First Record', 
              'Recorded By']
df

,0,1
0,Equatorial Diameter:,"6,792 km"
1,Polar Diameter:,"6,752 km"
2,Mass:,6.39 × 10^23 kg (0.11 Earths)
3,Moons:,2 (Phobos & Deimos)
4,Orbit Distance:,"227,943,824 km (1.38 AU)"
5,Orbit Period:,687 days (1.9 years)
6,Surface Temperature:,-87 to -5 °C
7,First Record:,2nd millennium BC
8,Recorded By:,Egyptian astronomers


In [617]:
url = 'https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars'
browser.visit(url)
time.sleep(3)

In [658]:
html = browser.html
soup = BeautifulSoup(html,'html.parser')
hemispheres = soup.find_all('div',class_='description')

hemi_image_list=[]
hemi_title_list=[]

for hemisphere in hemispheres:
    hemi_image = hemisphere.a['href']
#     hemi_image_link = (f'https://astrogeology.usgs.gov{hemi_image}')
    hemi_image_list.append(hemi_image)
    hemi_title = hemisphere.h3.text.strip()
    hemi_title_list.append(hemi_title)
    print(hemi_image)
    print(hemi_title)
for hemi_link in hemi_image_list:
    browser.click_link_by_href(hemi_link)

print(hemi_image_list)
print(hemi_title_list)


    
#     title_list.append(title)
#     para = str(article.find('div',class_='article_teaser_body').text.strip())
#     para_list.append(para)
#     print('______________')
#     print(title)
#     print(para)



/search/map/Mars/Viking/cerberus_enhanced
Cerberus Hemisphere Enhanced
/search/map/Mars/Viking/schiaparelli_enhanced
Schiaparelli Hemisphere Enhanced
/search/map/Mars/Viking/syrtis_major_enhanced
Syrtis Major Hemisphere Enhanced
/search/map/Mars/Viking/valles_marineris_enhanced
Valles Marineris Hemisphere Enhanced


ElementNotInteractableException: Message: element not interactable: element has zero size
  (Session info: chrome=85.0.4183.102)


In [645]:
for x in range(len(hemi_image_list)):
    browser.click_link_by_href(each_link)
    browser.click_link_by_text('Sample')



ElementNotInteractableException: Message: element not interactable: element has zero size
  (Session info: chrome=85.0.4183.102)
